In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

import matplotlib.pyplot as plt
import seaborn as sns

import env
import prepare
import acquire


# Titanic Dataset

In [2]:
# import titanic dataset, run our prepare functions, split into train/validate/test, and validate size of df's
df = acquire.get_titanic_data()
df = prepare.prep_titanic(df)
tit_train, tit_validate, tit_test = prepare.my_split(df, target='survived')
tit_train.shape, tit_validate.shape, tit_test.shape

((426, 15), (143, 15), (143, 15))

In [3]:
tit_train.head(1)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,embark_town,alone,sex_female,sex_male,embark_town_Cherbourg,embark_town_Queenstown,embark_town_Southampton
870,0,3,male,26.0,0,0,7.8958,S,Southampton,1,0,1,0,0,1


## What is your baseline prediction?  What is your baseline accuracy? 

In [4]:
# find the most common condition for our target to determine baseline.  It's rate of occurance is the accuracy of 
# our baseline
print('The most common value (our baseline) is:',tit_train.survived.value_counts().idxmax())
print('The rate of occurance (our baseline accuracy) is:', len(tit_train[tit_train.survived == 0]) / len(tit_train) * 100)

The most common value (our baseline) is: 0
The rate of occurance (our baseline accuracy) is: 59.624413145539904


# Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)

In [5]:
# split each of our datasets into X and y 

X_train = tit_train.drop(columns=['survived', 'sex', 'embarked', 'embark_town', 'sex_female', 'embark_town_Cherbourg'])
y_train = tit_train.survived

X_validate = tit_validate.drop(columns=['survived', 'sex', 'embarked', 'embark_town', 'sex_female', 'embark_town_Cherbourg'])
y_validate = tit_validate.survived

X_test = tit_test.drop(columns=['survived', 'sex', 'embarked', 'embark_town', 'sex_female', 'embark_town_Cherbourg'])
y_test = tit_test.survived

In [6]:
# create and fit the model
clf = DecisionTreeClassifier()

clf = clf.fit(X_train, y_train)

y_pred = clf.predict(X_train)
y_pred_prob = clf.predict_proba(X_train)


## Evaluate your in-sample results using the model score, confusion matrix, and classification report.

In [7]:
# model score
print(f'training score: {clf.score(X_train, y_train):.2%}')

training score: 99.30%


In [8]:
# confusion matrix
cm = confusion_matrix(y_train, y_pred)
print(pd.DataFrame(cm))

     0    1
0  254    0
1    3  169


In [9]:
# classification report
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99       254
           1       1.00      0.98      0.99       172

    accuracy                           0.99       426
   macro avg       0.99      0.99      0.99       426
weighted avg       0.99      0.99      0.99       426



## Compute: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [10]:
# accuracy
print('------- Model 1 -----------')
print('Accuracy score is: ', accuracy_score(y_train, y_pred))

------- Model 1 -----------
Accuracy score is:  0.9929577464788732


In [11]:
# true positive rate, false positive rate, true negative rate, false negative rate
tp = cm[0,0]
tn = cm[1,1]
fp = cm[0,1]
fn = cm[1,0]
print('------- Model 1 -----------')
print('True positive rate is:', tp/(tp+tn+fp+fn))
print('False positive rate is', fp/(tp+tn+fp+fn))
print('True negative rate is', tn/(tp+tn+fp+fn))
print('False negative rate is', fn/(tp+tn+fp+fn))

------- Model 1 -----------
True positive rate is: 0.596244131455399
False positive rate is 0.0
True negative rate is 0.3967136150234742
False negative rate is 0.007042253521126761


In [12]:
# precision, recall, f1-score and support
print('------- Model 1 -----------')
print('Precision is: ', precision_score(y_train, y_pred))
print('Recall is:', recall_score(y_train, y_pred))
print('f1 score is:', f1_score(y_train, y_pred))
print('Support is 0:', tp+fp)
print('           1:', tn+fn)

------- Model 1 -----------
Precision is:  1.0
Recall is: 0.9825581395348837
f1 score is: 0.9912023460410557
Support is 0: 254
           1: 172


## Run through steps 2-4 using a different max_depth value.

In [13]:
# changing max depth to 5
clf2 = DecisionTreeClassifier(max_depth=5)

clf2 = clf2.fit(X_train, y_train)

y_pred2 = clf2.predict(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [14]:
# model score
print('------- Model 2 -----------')
print(f'training score for model2: {clf2.score(X_train, y_train):.2%}')

------- Model 2 -----------
training score for model2: 85.68%


In [15]:
# confusion matrix
cm2 = confusion_matrix(y_train, y_pred2)
print(pd.DataFrame(cm2))

     0    1
0  230   24
1   37  135


In [16]:
# classification report
print('------- Model 2 -----------')
print(classification_report(y_train, y_pred2))

------- Model 2 -----------
              precision    recall  f1-score   support

           0       0.86      0.91      0.88       254
           1       0.85      0.78      0.82       172

    accuracy                           0.86       426
   macro avg       0.86      0.85      0.85       426
weighted avg       0.86      0.86      0.86       426



In [17]:
# accuracy
print('------- Model 2 -----------')
print('Accuracy score for model 2 is: ', accuracy_score(y_train, y_pred2))

------- Model 2 -----------
Accuracy score for model 2 is:  0.8568075117370892


In [18]:
# true positive rate, false positive rate, true negative rate, false negative rate
tp2 = cm2[0,0]
tn2 = cm2[1,1]
fp2 = cm2[0,1]
fn2 = cm2[1,0]

print('------- Model 2 -----------')
print('True positive rate is:', tp2/(tp2+tn2+fp2+fn2))
print('False positive rate is', fp2/(tp2+tn2+fp2+fn2))
print('True negative rate is', tn2/(tp2+tn2+fp2+fn2))
print('False negative rate is', fn2/(tp2+tn2+fp2+fn2))

------- Model 2 -----------
True positive rate is: 0.539906103286385
False positive rate is 0.056338028169014086
True negative rate is 0.31690140845070425
False negative rate is 0.08685446009389672


In [19]:
# precision, recall, f1-score and support
print('------- Model 2 -----------')
print('Precision is: ', precision_score(y_train, y_pred2))
print('Recall is:', recall_score(y_train, y_pred2))
print('f1 score is:', f1_score(y_train, y_pred2))
print('Support is 0:', tp2+fp2)
print('           1:', tn2+fn2)

------- Model 2 -----------
Precision is:  0.8490566037735849
Recall is: 0.7848837209302325
f1 score is: 0.8157099697885195
Support is 0: 254
           1: 172


## Which model performs better on your in-sample data?

Model 1 performs better when evaluating performance against the in-sample (training) data

## Which model performs best on your out-of-sample data, the validate set?

In [20]:
print('------- Model 1 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf.score(X_validate, y_validate)))
print('')
print('------- Model 2 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf2.score(X_validate, y_validate)))


------- Model 1 -----------
Accuracy of Decision Tree classifier on validate set: 0.76

------- Model 2 -----------
Accuracy of Decision Tree classifier on validate set: 0.82


In [21]:
# classification report
y_pred = clf.predict(X_validate)
y_pred2 = clf2.predict(X_validate)

print('------- Model 1 -----------')
print(classification_report(y_validate, y_pred))

print('------- Model 2 -----------')
print(classification_report(y_validate, y_pred2))

------- Model 1 -----------
              precision    recall  f1-score   support

           0       0.76      0.87      0.81        85
           1       0.76      0.60      0.67        58

    accuracy                           0.76       143
   macro avg       0.76      0.74      0.74       143
weighted avg       0.76      0.76      0.76       143

------- Model 2 -----------
              precision    recall  f1-score   support

           0       0.80      0.92      0.86        85
           1       0.85      0.67      0.75        58

    accuracy                           0.82       143
   macro avg       0.83      0.80      0.80       143
weighted avg       0.82      0.82      0.81       143



Model 2 performs slightly better with a max depth of 5

# Work through these same exercises using the Telco dataset.
    ## building two models simultaneously

In [22]:
# get and prepare iris data
iris = acquire.get_iris_data()
iris = prepare.prep_iris(iris)
i_train, i_validate, i_test = prepare.my_split(iris, target='species')
i_train.shape, i_validate.shape, i_test.shape

((90, 5), (30, 5), (30, 5))

In [23]:
# find the baseline and the baseline accuracy
baseline = i_train.species.value_counts().idxmax()
print('The most common value (our baseline) is:', baseline)
print('The rate of occurance (our baseline accuracy) is:', len(i_train[i_train.species == baseline]) / len(i_train) * 100)

The most common value (our baseline) is: setosa
The rate of occurance (our baseline accuracy) is: 33.33333333333333


In [24]:
# split each of our datasets into X and y 

X_train = i_train.drop(columns=('species'))
y_train = i_train.species

X_validate = i_validate.drop(columns=('species'))
y_validate = i_validate.species

X_test = i_test.drop(columns='species')
y_test = i_test.species

In [25]:
# create and fit the models
clf1 = DecisionTreeClassifier(max_depth=8)
clf2 = DecisionTreeClassifier(max_depth=4)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)

y_pred1 = clf1.predict(X_train)
y_pred2 = clf2.predict(X_train)

y_pred_prob1 = clf1.predict_proba(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [26]:
# model score
print(f'Model 1 training score: {clf1.score(X_train, y_train):.2%}')
print(f'Model 2 training score: {clf2.score(X_train, y_train):.2%}')

Model 1 training score: 100.00%
Model 2 training score: 100.00%


In [27]:
# confusion matrix
cm1 = confusion_matrix(y_train, y_pred1)
cm2 = confusion_matrix(y_train, y_pred2)
print('----Model 1----')
print(pd.DataFrame(cm1))
print('')
print('----Model 2----')
print(pd.DataFrame(cm2))

----Model 1----
    0   1   2
0  30   0   0
1   0  30   0
2   0   0  30

----Model 2----
    0   1   2
0  30   0   0
1   0  30   0
2   0   0  30


In [28]:
# classification report
print(classification_report(y_train, y_pred1))
print(classification_report(y_train, y_pred2))

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        30
  versicolor       1.00      1.00      1.00        30
   virginica       1.00      1.00      1.00        30

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90

              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        30
  versicolor       1.00      1.00      1.00        30
   virginica       1.00      1.00      1.00        30

    accuracy                           1.00        90
   macro avg       1.00      1.00      1.00        90
weighted avg       1.00      1.00      1.00        90



## ...and comparing with validation data

In [29]:
print('------- Model 1 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf1.score(X_validate, y_validate)))
print('')
print('------- Model 2 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf2.score(X_validate, y_validate)))

------- Model 1 -----------
Accuracy of Decision Tree classifier on validate set: 0.93

------- Model 2 -----------
Accuracy of Decision Tree classifier on validate set: 0.93


In [30]:
# classification report
y_pred1 = clf1.predict(X_validate)
y_pred2 = clf2.predict(X_validate)

print('------- Model 1 -----------')
print(classification_report(y_validate, y_pred1))

print('------- Model 2 -----------')
print(classification_report(y_validate, y_pred2))

------- Model 1 -----------
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.80      0.89        10
   virginica       0.83      1.00      0.91        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30

------- Model 2 -----------
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        10
  versicolor       1.00      0.80      0.89        10
   virginica       0.83      1.00      0.91        10

    accuracy                           0.93        30
   macro avg       0.94      0.93      0.93        30
weighted avg       0.94      0.93      0.93        30



The second model performs better with max depth of 4 versus max depth of 8 for the first model

# Experiment with this model on other datasets with a higher number of output classes.


In [31]:
# The glass dataset contains 9 columns quantifying the contents of 9 different elements in a glass sample.  
# The 10th column is the class of the glass, and integer from 1-7.

glass = pd.read_csv('glass.csv')
glass.head()

,1.52101,13.64,4.49,1.10,71.78,0.06,8.75,0.00,0.00.1,1
0,1.51761,13.89,3.60,1.36,72.73,0.48,7.83,0.0,0.00,1
1,1.51618,13.53,3.55,1.54,72.99,0.39,7.78,0.0,0.00,1
2,1.51766,13.21,3.69,1.29,72.61,0.57,8.22,0.0,0.00,1
3,1.51742,13.27,3.62,1.24,73.08,0.55,8.07,0.0,0.00,1
4,1.51596,12.79,3.61,1.62,72.97,0.64,8.07,0.0,0.26,1


In [32]:
glass.columns =['ri', 'na', 'mg', 'al', 'si', 'k','ca','ba','fe', 'class']

In [33]:
# split into train, validate, test
g_train, g_validate, g_test = prepare.my_split(glass, target='class')
g_train.shape, g_validate.shape, g_test.shape

((127, 10), (43, 10), (43, 10))

In [34]:
# find the baseline and the baseline accuracy
baseline = g_train['class'].value_counts().idxmax()
print('The most common value (our baseline) is:', baseline)
print('The rate of occurance (our baseline accuracy) is:', len(g_train[g_train['class'] == baseline]) / len(g_train) * 100)

The most common value (our baseline) is: 2
The rate of occurance (our baseline accuracy) is: 36.22047244094488


In [35]:
# split each of our datasets into X and y 

X_train = g_train.drop(columns=('class'))
y_train = g_train['class']

X_validate = g_validate.drop(columns=('class'))
y_validate = g_validate['class']

X_test = g_test.drop(columns='class')
y_test = g_test['class']

In [36]:
# create and fit the models
clf1 = DecisionTreeClassifier(max_depth=8)
clf2 = DecisionTreeClassifier(max_depth=4)

clf1 = clf1.fit(X_train, y_train)
clf2 = clf2.fit(X_train, y_train)

y_pred1 = clf1.predict(X_train)
y_pred2 = clf2.predict(X_train)

y_pred_prob1 = clf1.predict_proba(X_train)
y_pred_prob2 = clf2.predict_proba(X_train)

In [37]:
# model score
print(f'Model 1 training score: {clf1.score(X_train, y_train):.2%}')
print(f'Model 2 training score: {clf2.score(X_train, y_train):.2%}')

Model 1 training score: 98.43%
Model 2 training score: 77.95%


In [38]:
# confusion matrix
cm1 = confusion_matrix(y_train, y_pred1)
cm2 = confusion_matrix(y_train, y_pred2)
print('----Model 1----')
print(pd.DataFrame(cm1))
print('')
print('----Model 2----')
print(pd.DataFrame(cm2))

----Model 1----
    0   1  2  3  4   5
0  41   0  0  0  0   0
1   1  45  0  0  0   0
2   1   0  9  0  0   0
3   0   0  0  8  0   0
4   0   0  0  0  5   0
5   0   0  0  0  0  17

----Model 2----
    0   1  2  3  4   5
0  38   1  2  0  0   0
1  14  28  2  2  0   0
2   3   2  5  0  0   0
3   0   0  0  8  0   0
4   1   0  0  0  4   0
5   0   0  1  0  0  16


In [39]:
# classification report
print(classification_report(y_train, y_pred1))
print(classification_report(y_train, y_pred2))

              precision    recall  f1-score   support

           1       0.95      1.00      0.98        41
           2       1.00      0.98      0.99        46
           3       1.00      0.90      0.95        10
           5       1.00      1.00      1.00         8
           6       1.00      1.00      1.00         5
           7       1.00      1.00      1.00        17

    accuracy                           0.98       127
   macro avg       0.99      0.98      0.99       127
weighted avg       0.98      0.98      0.98       127

              precision    recall  f1-score   support

           1       0.68      0.93      0.78        41
           2       0.90      0.61      0.73        46
           3       0.50      0.50      0.50        10
           5       0.80      1.00      0.89         8
           6       1.00      0.80      0.89         5
           7       1.00      0.94      0.97        17

    accuracy                           0.78       127
   macro avg       0.81

In [40]:
print('------- Model 1 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf1.score(X_validate, y_validate)))
print('')
print('------- Model 2 -----------')
print('Accuracy of Decision Tree classifier on validate set: {:.2f}'
     .format(clf2.score(X_validate, y_validate)))

------- Model 1 -----------
Accuracy of Decision Tree classifier on validate set: 0.70

------- Model 2 -----------
Accuracy of Decision Tree classifier on validate set: 0.60


In [41]:
# classification reports
y_pred1 = clf1.predict(X_validate)
y_pred2 = clf2.predict(X_validate)

print('------- Model 1 -----------')
print(classification_report(y_validate, y_pred1))

print('------- Model 2 -----------')
print(classification_report(y_validate, y_pred2))

------- Model 1 -----------
              precision    recall  f1-score   support

           1       0.68      0.93      0.79        14
           2       0.83      0.67      0.74        15
           3       0.33      0.25      0.29         4
           5       0.33      0.50      0.40         2
           6       0.50      0.50      0.50         2
           7       1.00      0.67      0.80         6

    accuracy                           0.70        43
   macro avg       0.61      0.59      0.59        43
weighted avg       0.72      0.70      0.69        43

------- Model 2 -----------
              precision    recall  f1-score   support

           1       0.59      0.93      0.72        14
           2       0.71      0.33      0.45        15
           3       0.33      0.25      0.29         4
           5       0.40      1.00      0.57         2
           6       0.50      0.50      0.50         2
           7       1.00      0.67      0.80         6

    accuracy         

The first model (max depth 8) performed better on the training set, but the second model (max depth 4)
performed better on the validation set